In [15]:
import copy, numpy as np
np.random.seed(1)

def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

def sigmoid_output_to_derivative(output):#sigmoid的导数
    return output*(1 - output)

int2binary = {}
binary_dim = 8

larget_number = pow(2,binary_dim) #最大的位数是2的8次方
binary = np.unpackbits(np.array([range(larget_number)], dtype=np.uint8).T, axis = 1)
binary#构成一个二进制的数据集


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [16]:
for i in range(larget_number):
    int2binary[i] = binary[i]#顺序形成字典
    int2binary

In [17]:
#输入的变量
alpha = 0.1
input_dim = 2
hidden_dim = 16
output_dim = 1

#初始化中间层的参数
synapse_0 = 2 * np.random.random((input_dim, hidden_dim)) - 1
synapse_1 = 2 * np.random.random((hidden_dim, output_dim)) - 1
synapse_h = 2 * np.random.random((hidden_dim, hidden_dim)) - 1

synapse_0_update = np.zeros_like(synapse_0)# 相同shape的矩阵，值全部是0
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)
synapse_0_update

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
for j in range(10000):
    #生成简单的加法问题
    a_int = np.random.randint(larget_number/2)
    a = int2binary[a_int]
    
    b_int = np.random.randint(larget_number/2)
    b = int2binary[b_int]
    
    c_int = a_int+b_int
    c = int2binary[c_int]
    
    #先保留最好的结果
    d = np.zeros_like(c)
    
    overallError = 0
    
    layer_2_deltas = list()
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_dim))
    
    #move the along the positions in binary encoding
    for position in range(binary_dim):
        #产生输入和输出   关于这里为什么需要倒叙输出还有一些意味
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])#为什么这个输入要设置成这样
        y = np.array([[c[binary_dim - position - 1]]]).T
        
        #隐藏层的输入和输出
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))
        
        #输出层的输入和输出
        #print(np.dot(layer_1, synapse_1))
        layer_2 = sigmoid(np.dot(layer_1, synapse_1))
        
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error) * sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error[0])
        
        #解码估计，以便于后续何以输出,
        
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        #保存隐藏层，以便于后续可以用于下一个时间步，这个时间步可以一直使用
        layer_1_values.append(copy.deepcopy(layer_1))
    
    future_layer_1_delta = np.zeros(hidden_dim)
    #应该是进行反向传播
    for position in range(binary_dim):
        X = np.array([[a[position], b[position]]])
        layer_1 = layer_1_values[-position - 1]#其实就是倒叙输入
        prev_layer_1 = layer_1_values[-position - 2]#前一步的输入
        
        #输出层的误差
        layer_2_delta = layer_2_deltas[-position - 1]
        #隐藏层的误
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)        
        #开始更新权重
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)#转化成二维数组
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
        
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha
    
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    if j % 1000 == 0:
        print("Error",overallError)
        print("Pred",d)
        print("True",c)
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print ("------------")

Error [3.82296578]
Pred [1 1 1 1 1 1 1 1]
True [0 0 0 1 1 1 0 1]
12 + 17 = 255
------------
Error [3.86661582]
Pred [1 1 1 1 1 1 1 1]
True [1 0 0 1 0 1 1 1]
107 + 44 = 255
------------
Error [4.08237013]
Pred [1 1 1 1 1 0 1 1]
True [1 0 0 0 1 1 0 1]
60 + 81 = 251
------------
Error [3.85797955]
Pred [0 0 0 1 0 0 0 0]
True [1 0 1 0 0 1 0 0]
92 + 72 = 16
------------
Error [2.86477346]
Pred [0 1 0 0 0 0 0 0]
True [0 1 0 1 1 0 0 0]
4 + 84 = 64
------------
Error [1.47189865]
Pred [0 0 1 1 0 1 0 0]
True [0 0 1 1 0 1 0 0]
43 + 9 = 52
------------
Error [0.78305062]
Pred [0 1 1 0 0 0 1 0]
True [0 1 1 0 0 0 1 0]
38 + 60 = 98
------------
Error [0.663452]
Pred [0 1 1 1 0 1 1 1]
True [0 1 1 1 0 1 1 1]
111 + 8 = 119
------------
Error [0.41581614]
Pred [1 0 0 1 0 0 1 0]
True [1 0 0 1 0 0 1 0]
35 + 111 = 146
------------
Error [0.42098233]
Pred [1 0 0 0 0 1 1 1]
True [1 0 0 0 0 1 1 1]
76 + 59 = 135
------------
